In [1]:
import spacy
from spacy import displacy
import string
from itertools import chain

from nltk.corpus import movie_reviews as mr
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.classify import NaiveBayesClassifier as nbc
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import twitter_samples, stopwords
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk import FreqDist, classify, NaiveBayesClassifier

import re, string, random




NER = spacy.load("en_core_web_sm")

raw_text = open('data.txt').read()

text1= NER(raw_text)


for word in text1.ents:
    print(word.text,word.label_)

morning TIME
today DATE
2016 DATE
a transformative year DATE
2016 DATE
Twitter ORG
the third quarter in a row DATE
every single day DATE
this past year DATE
hundreds CARDINAL
Twitter PRODUCT
Twitter PRODUCT
2017 DATE
Twitter ORG
Twitter PERSON
Twitter PRODUCT
Twitter PRODUCT
Krista PERSON
John PERSON
2016 DATE
Twitter PRODUCT
four CARDINAL
Twitter PRODUCT
Twitter PRODUCT
Twitter PRODUCT
2016 DATE
today DATE
just three months
 DATE
Twitter PRODUCT
the past 10 years DATE
NFL ORG
the past 10 years DATE
Twitter PRODUCT
Jan Peterson PERSON
12:00 TIME
Twitter PRODUCT
Youssef PERSON
late last year DATE
a year DATE
Ed Ho PERSON
two years DATE
Keith Coleman PERSON
Jan Peterson PERSON
28:25 TIME
Twitter PRODUCT
first ORDINAL
Twitter PRODUCT
Twitter ORG
each quarter DATE
the fourth quarter DATE
Twitter PRODUCT
the end of the quarter DATE
second ORDINAL
2016 than 2015 DATE
prior years DATE
2016 DATE
2017 DATE
mid-January DATE
the fourth quarter
February DATE
35% to 40% PERCENT
January DATE
mid-Jan

In [2]:
displacy.render(text1,style="ent",jupyter=True)

In [18]:
def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)


In [19]:
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
text = twitter_samples.strings('tweets.20150430-223406.json')
tweet_tokens = twitter_samples.tokenized('positive_tweets.json')[0]

stop_words = stopwords.words('english')

positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []

for tokens in positive_tweet_tokens:
    positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

for tokens in negative_tweet_tokens:
    negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

all_pos_words = get_all_words(positive_cleaned_tokens_list)

freq_dist_pos = FreqDist(all_pos_words)
print(freq_dist_pos.most_common(10))

positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

positive_dataset = [(tweet_dict, "Positive")
                     for tweet_dict in positive_tokens_for_model]

negative_dataset = [(tweet_dict, "Negative")
                     for tweet_dict in negative_tokens_for_model]

dataset = positive_dataset + negative_dataset

random.shuffle(dataset)

train_data = dataset[:7000]
test_data = dataset[7000:]

classifier = NaiveBayesClassifier.train(train_data)

print("Accuracy is:", classify.accuracy(classifier, test_data))

print(classifier.show_most_informative_features(10))

custom_tweet = open('/Users/sreekarbathula/Downloads/data.txt').read()

custom_tokens = remove_noise(word_tokenize(custom_tweet))

print(custom_tweet, classifier.classify(dict([token, True] for token in custom_tokens)))

[(':)', 3691), (':-)', 701), (':d', 658), ('thanks', 388), ('follow', 357), ('love', 333), ('...', 290), ('good', 283), ('get', 263), ('thank', 253)]
Accuracy is: 0.9966666666666667
Most Informative Features
                      :( = True           Negati : Positi =   2049.6 : 1.0
                      :) = True           Positi : Negati =   1006.9 : 1.0
                     sad = True           Negati : Positi =     32.0 : 1.0
                follower = True           Positi : Negati =     21.4 : 1.0
                    poor = True           Negati : Positi =     18.0 : 1.0
                followed = True           Negati : Positi =     15.1 : 1.0
                    damn = True           Negati : Positi =     14.1 : 1.0
               wonderful = True           Positi : Negati =     13.9 : 1.0
                     via = True           Positi : Negati =     13.7 : 1.0
                  arrive = True           Positi : Negati =     13.5 : 1.0
None
Good morning everyone and thanks for 

<generator object get_all_words at 0x141d9a970>


In [4]:
for word in text1.ents:
    if word.label_=='ORG':
        print(word.text,)

Twitter
Twitter
NFL
Twitter
Twitter
Periscope
Twitter


In [6]:
for word in text1.ents:
    if word.label_=='DATE':
        print(word.text,)

today
2016
a transformative year
2016
the third quarter in a row
every single day
this past year
2017
2016
2016
today
just three months

the past 10 years
the past 10 years
late last year
a year
two years
each quarter
the fourth quarter
the end of the quarter
2016 than 2015
prior years
2016
2017
mid-January
the fourth quarter
February
January
mid-January
February
earlier

this year
2016
the past 10 years
every single year
2017
this year
10-year
